<a href="https://colab.research.google.com/github/AnanyaBhalla/GC-SAM/blob/main/fine_tune_SAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://youtu.be/83tnWs_YBRQ

**This notebook walks you through the process of fine-tuning a Segment Anything Model (SAM) using custom data**.
<p>

**What is SAM?**
<br>
SAM is an image segmentation model developed by Meta AI. It was trained over 11 billion segmentation masks from millions of images. It is designed to take human prompts, in the form of points, bounding boxes or even a text prompt describing what should be segmented.
<p>

**What are the key features of SAM?**
<br>

* **Zero-shot generalization:** SAM can be used to segment objects that it has never seen before, without the need for additional training.

* **Flexible prompting:** SAM can be prompted with a variety of input, including points, boxes, and text descriptions.

* **Real-time mask computation:** SAM can generate masks for objects in real time. This makes SAM ideal for applications where it is necessary to segment objects quickly, such as autonomous driving and robotics.

* **Ambiguity awareness:** SAM is aware of the ambiguity of objects in images. This means that SAM can generate masks for objects even when they are partially occluded or overlapping with other objects.

<p>

**How does SAM work?**
<br>
SAM works by first encoding the image into a high-dimensional vector representation. The prompt is encoded into a separate vector representation. The two vector representations are then combined and passed to a mask decoder, which outputs a mask for the object specified by the prompt.
<p>
The image encoder is a vision transformer (ViT-H) model, which is a large language model that has been pre-trained on a massive dataset of images. The prompt encoder is a simple text encoder that converts the input prompt into a vector representation. The mask decoder is a lightweight transformer model that predicts the object mask from the image and prompt embeddings.
<p>

**SAM paper:** https://arxiv.org/pdf/2304.02643.pdf​
<p>

**Link to the dataset used in this demonstration:** https://www.epfl.ch/labs/cvlab/data/data-em/
<br>Courtesy: EPFL
<p>

This code has been heavily adapted from this notebook but modified to work with a truly custom dataset where we have a bunch of images and binary masks.
https://github.com/NielsRogge/Transformers-Tutorials/blob/master/SAM/Fine_tune_SAM_(segment_anything)_on_a_custom_dataset.ipynb



In [1]:
# If you're using Google Colab, you can mount your folder with the images onto Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install the required libraries
#SAM
!pip install git+https://github.com/facebookresearch/segment-anything.git
#Transformers
!pip install -q git+https://github.com/huggingface/transformers.git
#Datasets to prepare data and monai if you want to use special loss functions
!pip install datasets
!pip install -q monai
#Patchify to divide large images into smaller patches for training. (Not necessary for smaller images)
!pip install patchify

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-lsgolndm
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-lsgolndm
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36590 sha256=555df9033c152f764774254aa1cd07091e1d1fa5524a314ab6aceed1323cac99
  Stored in directory: /tmp/pip-ephem-wheel-cache-0ko4u1o_/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542

In [ ]:

import os
import numpy as np
import matplotlib.pyplot as plt
import tifffile
from patchify import patchify  #Only to handle large images
import random
from scipy import ndimage
from PIL import Image
from transformers import SamModel, SamConfig, SamProcessor
from tqdm import tqdm
from statistics import mean
from datasets import Dataset
import torch
from torch.nn.functional import threshold, normalize
from torch.optim import Adam
import monai
from torch.utils.data import DataLoader

Load data into numpy arrays. Here we are importing a tiff stack containing 165 images of size 768x1024. You can use glob to import multiple images from a directory or any other method. The goal is to end up with a numpy array for images and an array for masks.

In [36]:
# Load tiff stack images and masks - EPFL dataset has 165 large images as tiff image stack
#large_images = tifffile.imread("training_images.tif")
#large_masks = tifffile.imread("training_masks.tiff")
# Google Colab version
large_images = tifffile.imread("/content/drive/MyDrive/sml/SAM/trial/training_images.tif")
large_masks = tifffile.imread("/content/drive/MyDrive/sml/SAM/trial/training_masks.tiff")

large_images.shape

(165, 768, 1024)

In [ ]:
# Directory containing the PNG images
image_directory = "/content/drive/MyDrive/sml/SAM/GC_fine_tuning/training/wsi_pngs"

# Read all PNG images from the directory
image_list = []
for filename in sorted(os.listdir(image_directory)):
    if filename.endswith(".png"):
        img_path = os.path.join(image_directory, filename)
        img = Image.open(img_path)
        img_array = np.array(img)
        image_list.append(img_array)

# Convert the list of images to a numpy array
large_images = np.array(image_list)

# Verify the shape of the loaded images
print(large_images.shape)

Now. let us divide these large images into smaller patches for training. We can use patchify or write custom code.

In [ ]:
#Desired patch size for smaller images and step size.
patch_size = 256
step = 256

In [ ]:
all_img_patches = []
for img in range(large_images.shape[0]):
    large_image = large_images[img]
    patches_img = patchify(large_image, (patch_size, patch_size), step=step)  #Step=256 for 256 patches means no overlap

    for i in range(patches_img.shape[0]):
        for j in range(patches_img.shape[1]):

            single_patch_img = patches_img[i,j,:,:]
            all_img_patches.append(single_patch_img)

images = np.array(all_img_patches)

#Let us do the same for masks
all_mask_patches = []
for img in range(large_masks.shape[0]):
    large_mask = large_masks[img]
    patches_mask = patchify(large_mask, (patch_size, patch_size), step=step)  #Step=256 for 256 patches means no overlap

    for i in range(patches_mask.shape[0]):
        for j in range(patches_mask.shape[1]):

            single_patch_mask = patches_mask[i,j,:,:]
            single_patch_mask = (single_patch_mask / 255.).astype(np.uint8)
            all_mask_patches.append(single_patch_mask)

masks = np.array(all_mask_patches)

images.shape

In [35]:
def load_images_from_folder(folder, patch_size, step):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".png"):
            img = Image.open(os.path.join(folder, filename))
            img = np.array(img)
        elif filename.endswith(".tiff") or filename.endswith(".tif"):
            img = tifffile.imread(os.path.join(folder, filename))
        else:
            continue

        # Ensure image has 2 dimensions
        if img.ndim == 3 and img.shape[2] == 1:
            img = img.squeeze(axis=2)

        # Patchify the image
        patches_img = patchify(img, (patch_size, patch_size), step=step)
        for i in range(patches_img.shape[0]):
            for j in range(patches_img.shape[1]):
                single_patch_img = patches_img[i, j, :, :]
                images.append(single_patch_img)

    return np.array(images)

# Example usage:
folder_path = "/content/drive/MyDrive/sml/SAM/GC_fine_tuning/training/wsi_pngs"  # Your folder path
patch_size = 256
step = 256

all_img_patches = load_images_from_folder(folder_path, patch_size, step)
print(all_img_patches.shape)


ValueError: `window_shape` is incompatible with `arr_in.shape`

Now, let us delete empty masks as they may cause issues later on during training. If a batch contains empty masks then the loss function will throw an error as it may not know how to handle empty tensors.

In [ ]:
# Create a list to store the indices of non-empty masks
valid_indices = [i for i, mask in enumerate(masks) if mask.max() != 0]
# Filter the image and mask arrays to keep only the non-empty pairs
filtered_images = images[valid_indices]
filtered_masks = masks[valid_indices]
print("Image shape:", filtered_images.shape)  # e.g., (num_frames, height, width, num_channels)
print("Mask shape:", filtered_masks.shape)

In [ ]:
#load the pathologist's annotations and the agreement of the pathologists' annotations
swapnil_json=annotation_dir+"/json-swapnil/"+imagename+".json"
aekta_json=annotation_dir+'/json-aekta/'+imagename+'.json'
pat_json=annotation_dir+'/json-pat/'+imagename+'.json'
fangfang_json=annotation_dir+'/json-fangfang-2/'+imagename+'.json'
# Load annotations
def get_annotations(json_path,label):
    with open(json_path) as jsonFile:
        j=json.load(jsonFile)
    test=[]
    for a in j:
        try:
            x=len(a['properties']['classification'])
        except Exception as e:
            continue
        if a['properties']['classification']['name']==label:
            if a['geometry']['type']=="Polygon":
                for a2 in a['geometry']['coordinates']:
                    a2=[[int(i[0]),int(i[1])] for i in a2]
                    test.append(a2)
            elif a['geometry']['type']=="MultiPolygon":
                for a2 in a['geometry']['coordinates']:
                    for a3 in a2:
                        a3=[[int(i[0]),int(i[1])] for i in a3]
                        test.append(a3)
            elif a['geometry']['type']=="LineString":
                a2=a['geometry']['coordinates']
                a2=[[int(i[0]),int(i[1])] for i in a2]
                test.append(a2)
    return test
def load_annotations(json_path,label,magnification,DOWNSAMPLE,wsi_path):
    test1=get_annotations(json_path,label=label)
    annotations1={}
    annotations1[0]=test1
    wsi_1=slide.Slide(wsi_path,mag=magnification,annotations=annotations1)
    new_dims1=wsi_1.level_dimensions[magnification]
    new_dims1=(int(new_dims1[0]/DOWNSAMPLE ),int(new_dims1[1]/DOWNSAMPLE ))
    mask_1=wsi_1.slide_mask(resize=new_dims1,size=new_dims1)
    del wsi_1
    return mask_1
#prediction_path=[p for p in predictions if imagename in p][0]
#prediction=cv2.imread(prediction_path)
wsi_path=os.path.join(WSI_dir,imagename+".ndpi")
mask_1=load_annotations(aekta_json,'Other',magnification_level,DOWNSAMPLE,wsi_path) ## aekta's annotation
mask_2=load_annotations(swapnil_json,'germinal centre',magnification_level,DOWNSAMPLE,wsi_path)## Swapnil's annotation
mask_3=load_annotations(pat_json,'GC',magnification_level,DOWNSAMPLE,wsi_path)## Pat's annotation
mask_4=load_annotations(fangfang_json,'GC',magnification_level,DOWNSAMPLE,wsi_path)## Fangfang's annotation

MODIFIED: This function converts an image from Greyscale to RGB as `transformers.py` requires 3 channels (Greyscale images only contain 1 channel)

In [ ]:
def convert_to_rgb(image):
    """
    Converts a grayscale image to RGB by duplicating the single channel across three channels.

    Args:
        image (PIL.Image): The input grayscale image.

    Returns:
        PIL.Image: The converted RGB image.
    """
    if image.mode == 'L':  # Check if the image is grayscale
        image = np.array(image)
        image = np.stack((image,) * 3, axis=-1)  # Duplicate the single channel across three channels
        image = Image.fromarray(image, mode='RGB')
    return image

Let us create a 'dataset' that serves us input images and masks for the rest of our journey.

In [ ]:
# Convert the NumPy arrays to Pillow images and preprocess them
dataset_dict = {
    # MODIFIED
    "image": [convert_to_rgb(Image.fromarray(img)) for img in filtered_images],
    #"image": [Image.fromarray(img) for img in filtered_images],
    "label": [Image.fromarray(mask) for mask in filtered_masks],
}

# Create the dataset using the datasets.Dataset class
dataset = Dataset.from_dict(dataset_dict)


In [ ]:
# Function to print the number of channels in the first image
def print_number_of_channels(dataset):
    if len(dataset) > 0:
        item = dataset[0]
        image = item['image']
        # Convert to NumPy array if necessary
        if isinstance(image, Image.Image):
            image = np.array(image)
        num_channels = image.shape[-1] if image.ndim == 3 else 1
        print(f"The first image has {num_channels} channel(s)")
    else:
        print("The dataset is empty")

# Print the number of channels for the first image in the dataset
print_number_of_channels(dataset)

In [ ]:
dataset

Let us make sure out images and masks (labels) are loading appropriately

In [ ]:
img_num = random.randint(0, filtered_images.shape[0]-1)
example_image = dataset[img_num]["image"]
example_mask = dataset[img_num]["label"]

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Plot the first image on the left
axes[0].imshow(np.array(example_image), cmap='gray')  # Assuming the first image is grayscale
axes[0].set_title("Image")

# Plot the second image on the right
axes[1].imshow(example_mask, cmap='gray')  # Assuming the second image is grayscale
axes[1].set_title("Mask")

# Hide axis ticks and labels
for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

# Display the images side by side
plt.show()

Get bounding boxes from masks. You can get here directly if you are working with coco style annotations where bounding boxes are captured in a JSON file.

In [ ]:
#Get bounding boxes from mask.
def get_bounding_box(ground_truth_map):
  # get bounding box from mask
  y_indices, x_indices = np.where(ground_truth_map > 0)
  x_min, x_max = np.min(x_indices), np.max(x_indices)
  y_min, y_max = np.min(y_indices), np.max(y_indices)
  # add perturbation to bounding box coordinates
  H, W = ground_truth_map.shape
  x_min = max(0, x_min - np.random.randint(0, 20))
  x_max = min(W, x_max + np.random.randint(0, 20))
  y_min = max(0, y_min - np.random.randint(0, 20))
  y_max = min(H, y_max + np.random.randint(0, 20))
  bbox = [x_min, y_min, x_max, y_max]

  return bbox

In [ ]:
from torch.utils.data import Dataset

class SAMDataset(Dataset):
  """
  This class is used to create a dataset that serves input images and masks.
  It takes a dataset and a processor as input and overrides the __len__ and __getitem__ methods of the Dataset class.
  """
  def __init__(self, dataset, processor):
    self.dataset = dataset
    self.processor = processor

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    item = self.dataset[idx]
    image = item["image"]
    ground_truth_mask = np.array(item["label"])

    # get bounding box prompt
    prompt = get_bounding_box(ground_truth_mask)

    # prepare image and prompt for the model
    inputs = self.processor(image, input_boxes=[[prompt]], return_tensors="pt")

    # remove batch dimension which the processor adds by default
    inputs = {k:v.squeeze(0) for k,v in inputs.items()}

    # add ground truth segmentation
    inputs["ground_truth_mask"] = ground_truth_mask

    return inputs

In [ ]:
# Initialize the processor
processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

# Create an instance of the SAMDataset
train_dataset = SAMDataset(dataset=dataset, processor=processor)

example = train_dataset[0]
for k,v in example.items():
  print(k,v.shape)

In [ ]:
# Create a DataLoader instance for the training dataset
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, drop_last=False)

batch = next(iter(train_dataloader))
for k,v in batch.items():
  print(k,v.shape)

In [ ]:
batch["ground_truth_mask"].shape

In [ ]:
# Load the model
model = SamModel.from_pretrained("facebook/sam-vit-base")

# make sure we only compute gradients for mask decoder
for name, param in model.named_parameters():
  if name.startswith("vision_encoder") or name.startswith("prompt_encoder"):
    param.requires_grad_(False)

In [ ]:
# Initialize the optimizer and the loss function
optimizer = Adam(model.mask_decoder.parameters(), lr=1e-5, weight_decay=0)
#Try DiceFocalLoss, FocalLoss, DiceCELoss
seg_loss = monai.losses.DiceCELoss(sigmoid=True, squared_pred=True, reduction='mean')

In [ ]:
#Training loop
num_epochs = 1

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model.train()
for epoch in range(num_epochs):
    epoch_losses = []
    for batch in tqdm(train_dataloader):
      # forward pass
      outputs = model(pixel_values=batch["pixel_values"].to(device),
                      input_boxes=batch["input_boxes"].to(device),
                      multimask_output=False)

      # compute loss
      predicted_masks = outputs.pred_masks.squeeze(1)
      ground_truth_masks = batch["ground_truth_mask"].float().to(device)
      loss = seg_loss(predicted_masks, ground_truth_masks.unsqueeze(1))

      # backward pass (compute gradients of parameters w.r.t. loss)
      optimizer.zero_grad()
      loss.backward()

      # optimize
      optimizer.step()
      epoch_losses.append(loss.item())

    print(f'EPOCH: {epoch}')
    print(f'Mean loss: {mean(epoch_losses)}')

In [ ]:
# Save the model's state dictionary to a file
#torch.save(model.state_dict(), "first_model_checkpoint.pth")
# Colab version
torch.save(model.state_dict(), "/content/drive/MyDrive/sml/SAM/trial/first_model_checkpoint.pth")

**Inference**

In [ ]:
# Load the model configuration
model_config = SamConfig.from_pretrained("facebook/sam-vit-base")
processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

# Create an instance of the model architecture with the loaded configuration
my_mito_model = SamModel(config=model_config)
#Update the model by loading the weights from saved file.
#my_mito_model.load_state_dict(torch.load("first_model_checkpoint.pth"))
# Colab version
my_mito_model.load_state_dict(torch.load("/content/drive/MyDrive/sml/SAM/trial/first_model_checkpoint.pth"))

In [ ]:
# set the device to cuda if available, otherwise use cpu
device = "cuda" if torch.cuda.is_available() else "cpu"
my_mito_model.to(device)

In [ ]:
# let's take a random training example
idx = random.randint(0, filtered_images.shape[0]-1)

# load image
test_image = dataset[idx]["image"]

# get box prompt based on ground truth segmentation map
ground_truth_mask = np.array(dataset[idx]["label"])
prompt = get_bounding_box(ground_truth_mask)

# prepare image + box prompt for the model
inputs = processor(test_image, input_boxes=[[prompt]], return_tensors="pt")

# Move the input tensor to the GPU if it's not already there
inputs = {k: v.to(device) for k, v in inputs.items()}

my_mito_model.eval()

# forward pass
with torch.no_grad():
    outputs = my_mito_model(**inputs, multimask_output=False)

# apply sigmoid
medsam_seg_prob = torch.sigmoid(outputs.pred_masks.squeeze(1))
# convert soft mask to hard mask
medsam_seg_prob = medsam_seg_prob.cpu().numpy().squeeze()
medsam_seg = (medsam_seg_prob > 0.5).astype(np.uint8)


fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot the first image on the left
axes[0].imshow(np.array(test_image), cmap='gray')  # Assuming the first image is grayscale
axes[0].set_title("Image")

# Plot the second image on the right
axes[1].imshow(medsam_seg, cmap='gray')  # Assuming the second image is grayscale
axes[1].set_title("Mask")

# Plot the second image on the right
axes[2].imshow(medsam_seg_prob)  # Assuming the second image is grayscale
axes[2].set_title("Probability Map")

# Hide axis ticks and labels
for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

# Display the images side by side
plt.show()


Now, let us load a new image and segment it using our trained model. NOte that we need to provide some prompt. Since we do not know where the objects are going to be we cannot supply bounding boxes. So let us provide a grid of points as our prompt.

In [ ]:
#Apply a trained model on large image
#large_test_images = tifffile.imread("test_images.tiff")
#Colab version
large_test_images = tifffile.imread("/content/drive/MyDrive/sml/SAM/trial/test_images.tiff")

large_test_image = large_test_images[1]
patches = patchify(large_test_image, (256, 256), step=256)  #Step=256 for 256 patches means no overlap

In [ ]:
"""
input_points (torch.FloatTensor of shape (batch_size, num_points, 2)) —
Input 2D spatial points, this is used by the prompt encoder to encode the prompt.
Generally yields to much better results. The points can be obtained by passing a
list of list of list to the processor that will create corresponding torch tensors
of dimension 4. The first dimension is the image batch size, the second dimension
is the point batch size (i.e. how many segmentation masks do we want the model to
predict per input point), the third dimension is the number of points per segmentation
mask (it is possible to pass multiple points for a single mask), and the last dimension
is the x (vertical) and y (horizontal) coordinates of the point. If a different number
of points is passed either for each image, or for each mask, the processor will create
“PAD” points that will correspond to the (0, 0) coordinate, and the computation of the
embedding will be skipped for these points using the labels.

"""
# Define the size of your array
array_size = 256

# Define the size of your grid
grid_size = 10

# Generate the grid points
x = np.linspace(0, array_size-1, grid_size)
y = np.linspace(0, array_size-1, grid_size)

# Generate a grid of coordinates
xv, yv = np.meshgrid(x, y)

# Convert the numpy arrays to lists
xv_list = xv.tolist()
yv_list = yv.tolist()

# Combine the x and y coordinates into a list of list of lists
input_points = [[[int(x), int(y)] for x, y in zip(x_row, y_row)] for x_row, y_row in zip(xv_list, yv_list)]

#We need to reshape our nxn grid to the expected shape of the input_points tensor
# (batch_size, point_batch_size, num_points_per_image, 2),
# where the last dimension of 2 represents the x and y coordinates of each point.
#batch_size: The number of images you're processing at once.
#point_batch_size: The number of point sets you have for each image.
#num_points_per_image: The number of points in each set.
input_points = torch.tensor(input_points).view(1, 1, grid_size*grid_size, 2)

In [ ]:
print(np.array(input_points).shape)

In [ ]:
patches.shape

In [ ]:
# Select a random patch for segmentation

# Compute the total number of 256x256 arrays
#num_arrays = patches.shape[0] * patches.shape[1]
# Select a random index
#index = np.random.choice(num_arrays)
# Compute the indices in the original array
#i = index // patches.shape[1]
#j = index % patches.shape[1]

#Or pick a specific patch for study.
i, j = 1, 2

# Selected patch for segmentation
random_array = patches[i, j]

single_patch = Image.fromarray(random_array)
# prepare image for the model

# Check and reshape the single_patch to have 3 dimensions
if single_patch.mode == 'L':  # If it's a grayscale image
    single_patch = np.array(single_patch)
    single_patch = np.expand_dims(single_patch, axis=-1)  # Expand dimensions to (height, width, 1)
    single_patch = np.concatenate([single_patch] * 3, axis=-1)  # Duplicate the channel to get (height, width, 3)

# Convert the array back to PIL Image for the processor
single_patch = Image.fromarray(np.uint8(single_patch))


#First try without providing any prompt (no bounding box or input_points)
#inputs = processor(single_patch, return_tensors="pt")
#Now try with bounding boxes. Remember to uncomment.
inputs = processor(single_patch, input_points=input_points, return_tensors="pt")

# Move the input tensor to the GPU if it's not already there
inputs = {k: v.to(device) for k, v in inputs.items()}
my_mito_model.eval()


# forward pass
with torch.no_grad():
  outputs = my_mito_model(**inputs, multimask_output=False)

# apply sigmoid
single_patch_prob = torch.sigmoid(outputs.pred_masks.squeeze(1))
# convert soft mask to hard mask
single_patch_prob = single_patch_prob.cpu().numpy().squeeze()
single_patch_prediction = (single_patch_prob > 0.5).astype(np.uint8)


fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot the first image on the left
axes[0].imshow(np.array(single_patch), cmap='gray')  # Assuming the first image is grayscale
axes[0].set_title("Image")

# Plot the second image on the right
axes[1].imshow(single_patch_prob)  # Assuming the second image is grayscale
axes[1].set_title("Probability Map")

# Plot the second image on the right
axes[2].imshow(single_patch_prediction, cmap='gray')  # Assuming the second image is grayscale
axes[2].set_title("Prediction")

# Hide axis ticks and labels
for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

# Display the images side by side
plt.show()